In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from ClientCredentials import client_id, client_secret

In [2]:
# Logging in
username = "113586775"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [18]:
# Which playlists do I have?
playlists = spotify.user_playlists(username)
print(type(playlists))
print(playlists.keys())
print()
for i, p in enumerate(playlists['items']):
    print(f"{i+playlists['offset']}   {p['uri']}   {p['name']}")

<class 'dict'>
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

0   spotify:playlist:2tvrDqnp3vJd6l6Cw38bVZ   Favorites
1   spotify:playlist:367BXl0oqvDSbBu34T6OBH   Queen and other classics
2   spotify:playlist:4h69xgBmpi8WnrCzNG0saL   John Mayer
3   spotify:playlist:37i9dQZEVXbMDoHDwVN2tF   Global Top 50
4   spotify:playlist:2k19D3M2QSz3sdS0MzocSs   Rock
5   spotify:playlist:1Vs71OxUMfNXTLtp4USgTA   Party
6   spotify:playlist:5na5MdXxNrFa7JUUwg14qP   Blues
7   spotify:playlist:4qa6CoQZqpUun4Divl1Cjg   Piano
8   spotify:playlist:37i9dQZF1DXcSNirXzsHRX   Rock du kender
9   spotify:playlist:4F3VK81yAvFlVquazfxVLG   Tolkiens The Lord of the Rings, and The Hobbit - Complete Symphony


In [19]:
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

HTTP Error for GET to https://api.spotify.com/v1/search returned 404 due to Not found.


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/search?q=year%3A2018&limit=50&offset=2000&type=track:
 Not found., reason: None